In [ ]:
import warnings
from cryptography.utils import CryptographyDeprecationWarning
warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)  # Suppress specific deprecation warnings

import tkinter as tk  # Import tkinter for GUI creation
from tkinter import filedialog, Label, messagebox  # Import specific tkinter functions for file dialog, label, and messagebox
from PIL import Image, ImageTk  # Import PIL for image processing
import cv2  # Import OpenCV for image processing
import numpy as np  # Import numpy for numerical operations
from skimage.feature import hog  # Import HOG feature extractor from skimage
from skimage import exposure  # Import exposure module from skimage for image adjustments
import os  # Import os for file and directory operations

class SegmentationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Segmentation and Feature Extraction GUI")  # Set window title
        self.root.geometry("1200x800")  # Set fixed window size
        self.root.resizable(False, False)  # Make window non-resizable

        self.setup_layout()  # Call the method to set up the GUI layout
        self.image = None  # Placeholder for the input image
        self.processed_image = None  # Placeholder for the processed image
        self.segmented_images = []  # List to store segmented images
        self.hog_image = None  # Placeholder for the HOG image

    def setup_layout(self):
        # Frame for buttons on the left
        self.button_frame = tk.Frame(self.root, width=200)
        self.button_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)

        # Buttons for various functionalities
        self.import_btn = tk.Button(self.button_frame, text="Load Image", command=self.load_image)
        self.import_btn.pack(fill=tk.X, pady=5)
        
        self.process_btn = tk.Button(self.button_frame, text="Process Image", command=self.process_image)
        self.process_btn.pack(fill=tk.X, pady=5)

        self.segment_btn = tk.Button(self.button_frame, text="Segment Image", command=self.segment_image)
        self.segment_btn.pack(fill=tk.X, pady=5)

        self.evaluate_btn = tk.Button(self.button_frame, text="Evaluate Segmentation", command=self.evaluate_and_save)
        self.evaluate_btn.pack(fill=tk.X, pady=5)

        self.hog_btn = tk.Button(self.button_frame, text="Display HoG Features", command=self.display_hog_features)
        self.hog_btn.pack(fill=tk.X, pady=5)

        self.save_hog_btn = tk.Button(self.button_frame, text="Save HoG Picture", command=self.save_hog_picture)
        self.save_hog_btn.pack(fill=tk.X, pady=5)

        self.clear_btn = tk.Button(self.button_frame, text="Clear All", command=self.clear_all)
        self.clear_btn.pack(fill=tk.X, pady=5)

        self.exit_btn = tk.Button(self.button_frame, text="Exit", command=self.exit_app)
        self.exit_btn.pack(fill=tk.X, pady=5)

        # Image display area in the center
        self.image_frame = tk.Frame(self.root)
        self.image_frame.pack(side=tk.LEFT, expand=True, padx=10, pady=10)

        self.labels = []  # List to store label widgets for image display
        self.images = []  # List to store images

        for i in range(9):  # Create 9 labels to fit more images
            label_frame = tk.Frame(self.image_frame, width=250, height=250, bg='gray')
            label_frame.grid_propagate(False)  # Prevent frame from resizing to fit content
            label_frame.grid(row=i//3, column=i%3, padx=5, pady=5)
            label = tk.Label(label_frame)
            label.pack(fill=tk.BOTH, expand=True)
            self.labels.append(label)  # Append label to the labels list

    def load_image(self):
        file_path = filedialog.askopenfilename()  # Open file dialog to select an image
        if file_path:
            self.image = cv2.imread(file_path)  # Read the image using OpenCV
            self.display_image(self.image, self.labels[0], "Input Image")  # Display the input image in the first label

    def display_image(self, img, label, text=""):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert the image from BGR to RGB
        img = Image.fromarray(img)  # Convert the image to a PIL image
        img = img.resize((200, 200), Image.LANCZOS)  # Resize the image to fit the label
        img = ImageTk.PhotoImage(img)  # Convert the PIL image to an ImageTk object
        label.config(image=img)  # Set the image on the label
        label.image = img  # Keep a reference to the image to prevent garbage collection
        label.config(text=text, compound='top')  # Set the text on the label

    def process_image(self):
        if self.image is not None:
            # Convert to Grayscale
            gray_image = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
            self.display_image(cv2.cvtColor(gray_image, cv2.COLOR_GRAY2RGB), self.labels[1], "Grayscale")

            # Enhance Image
            enhanced_image = cv2.equalizeHist(gray_image)  # Equalize the histogram
            enhanced_image = cv2.GaussianBlur(enhanced_image, (5, 5), 0)  # Apply Gaussian blur
            self.display_image(cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2RGB), self.labels[2], "Enhanced")

            # Remove Hairlines
            kernel = np.ones((5, 5), np.uint8)  # Create a kernel for morphological operations
            hair_removed_image = cv2.morphologyEx(enhanced_image, cv2.MORPH_CLOSE, kernel)  # Apply morphological close operation
            self.display_image(cv2.cvtColor(hair_removed_image, cv2.COLOR_GRAY2RGB), self.labels[3], "Hair Removed")

            self.processed_image = hair_removed_image  # Store the processed image

    def segment_image(self):
        if self.processed_image is not None:
            # Otsu Segmentation
            _, otsu_segmented = cv2.threshold(self.processed_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            self.segmented_images.append(('Otsu', otsu_segmented))  # Append the segmented image to the list
            self.display_image(cv2.cvtColor(otsu_segmented, cv2.COLOR_GRAY2RGB), self.labels[4], "Otsu Segmentation")

            # K-means Segmentation
            Z = self.processed_image.reshape((-1, 1))  # Reshape the image to a 1D array
            Z = np.float32(Z)  # Convert to float32
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)  # Define criteria for K-means
            K = 2  # Number of clusters
            _, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
            center = np.uint8(center)  # Convert centers to uint8
            res = center[label.flatten()]  # Assign cluster centers to each pixel
            kmeans_segmented = res.reshape((self.processed_image.shape))  # Reshape back to the original image shape
            _, kmeans_segmented_binary = cv2.threshold(kmeans_segmented, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            self.segmented_images.append(('K-means', kmeans_segmented_binary))  # Append the segmented image to the list
            self.display_image(cv2.cvtColor(kmeans_segmented_binary, cv2.COLOR_GRAY2RGB), self.labels[5], "K-means Segmentation")

            # Canny Segmentation
            canny_segmented = cv2.Canny(self.processed_image, 0, 225)  # Apply Canny edge detection
            self.segmented_images.append(('Canny', canny_segmented))  # Append the segmented image to the list
            self.display_image(cv2.cvtColor(canny_segmented, cv2.COLOR_GRAY2RGB), self.labels[6], "Canny Segmentation")

    def display_hog_features(self):
        if self.processed_image is not None:
            fd, hog_image = hog(self.processed_image, orientations=9, pixels_per_cell=(8, 8),
                                cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')  # Compute HOG features
            hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))  # Rescale the intensity of the HOG image
            self.hog_image = (hog_image_rescaled * 255).astype(np.uint8)  # Convert HOG image to uint8
            self.display_image(cv2.cvtColor(self.hog_image, cv2.COLOR_GRAY2RGB), self.labels[7], "HOG Features")  # Display the HOG image

    def save_hog_picture(self):
        if self.hog_image is not None:
            hog_dir = "HOG_Images"  # Directory to save HOG images
            if not os.path.exists(hog_dir):
                os.makedirs(hog_dir)  # Create directory if it does not exist
            save_path = os.path.join(hog_dir, "hog_image.png")  # Define the save path for the HOG image
            try:
                if cv2.imwrite(save_path, self.hog_image):  # Save the HOG image
                    messagebox.showinfo("Result", f"HoG picture saved as {save_path}")  # Show success message
                else:
                    messagebox.showerror("Error", "Failed to save HoG image")  # Show error message
            except Exception as e:
                print(f"Error during save: {e}")  # Print exception if any
        else:
            messagebox.showwarning("Warning", "No HoG image to save")  # Show warning if no HOG image is available
            
    def evaluate_and_save(self):
        if self.segmented_images and self.image is not None:
            ground_truth = filedialog.askopenfilename(title="Select Ground Truth Mask")  # Open file dialog to select ground truth mask
            if ground_truth:
                ground_truth = cv2.imread(ground_truth, cv2.IMREAD_GRAYSCALE)  # Read the ground truth mask in grayscale
                best_metric = -1  # Initialize the best metric
                best_image = None  # Placeholder for the best segmented image
                best_name = ""  # Placeholder for the name of the best segmentation method
                
                for name, segmented in self.segmented_images:  # Iterate through segmented images
                    metrics = self.evaluate_segmentation(segmented, ground_truth)  # Evaluate segmentation metrics
                    if metrics['dice'] > best_metric:  # Compare metrics to find the best segmentation
                        best_metric = metrics['dice']
                        best_image = segmented
                        best_name = name
                        
                if best_image is not None:
                    seg_dir = "SegmentedImages"  # Directory to save segmented images
                    if not os.path.exists(seg_dir):
                        os.makedirs(seg_dir)  # Create directory if it does not exist
                    save_path = os.path.join(seg_dir, f"best_segmentation_{best_name}.png")  # Define the save path for the best segmented image
                    try:
                        if cv2.imwrite(save_path, best_image):  # Save the best segmented image
                            messagebox.showinfo("Result", f"Best segmentation saved as {save_path}")  # Show success message
                        else:
                            messagebox.showerror("Error", "Failed to save segmentation image")  # Show error message
                    except Exception as e:
                        print(f"Error during save: {e}")  # Print exception if any

    def evaluate_segmentation(self, predicted, ground_truth):
        TP = np.sum((predicted == 255) & (ground_truth == 255))  # Calculate True Positives
        TN = np.sum((predicted == 0) & (ground_truth == 0))  # Calculate True Negatives
        FP = np.sum((predicted == 255) & (ground_truth == 0))  # Calculate False Positives
        FN = np.sum((predicted == 0) & (ground_truth == 255))  # Calculate False Negatives

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0  # Calculate Precision
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0  # Calculate Recall
        accuracy = (TP + TN) / (TP + TN + FP + FN)  # Calculate Accuracy
        dice = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0  # Calculate Dice Coefficient
        jaccard = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0  # Calculate Jaccard Index

        return {
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'dice': dice,
            'jaccard': jaccard
        }  # Return the metrics as a dictionary

    def clear_all(self):
        for label in self.labels:  # Iterate through all labels
            label.configure(image='', text='')  # Clear image and text from labels
        self.filepath = None  # Clear the file path
        self.images.clear()  # Clear the images list

    def exit_app(self):
        self.root.destroy()  # Destroy the main window to exit the application

if __name__ == "__main__":
    root = tk.Tk()  # Create the main window
    app = SegmentationApp(root)  # Create an instance of the SegmentationApp
    root.mainloop()  # Run the main loop to keep the GUI window open


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Smriti\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Smriti\AppData\Local\Temp\ipykernel_15336\3920404528.py", line 76, in load_image
    self.display_image(self.image, self.labels[0], "Input Image")  # Display the input image in the first label
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Smriti\AppData\Local\Temp\ipykernel_15336\3920404528.py", line 79, in display_image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert the image from BGR to RGB
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

